In [1]:
# Import the dependencies.
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from citipy import citipy
import requests
from config import weather_api_key
from datetime import datetime

In [2]:
# Starting URL for Weather Map API Call.
url = "http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=" + weather_api_key
print(url)

http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=6faa1d1863e2f802e364d3f5294d3587


In [3]:
# Create a set of random latitude and longitude combinations.
lats = np.random.uniform(low=-90.000, high=90.000, size=1500)
lngs = np.random.uniform(low=-180.000, high=180.000, size=1500)
lat_lngs = zip(lats, lngs)
lat_lngs

In [4]:
# Add the latitudes and longitudes to a list.
coordinates = list(lat_lngs)

In [5]:
# Create a list for holding the cities.
cities = []
# Identify the nearest city for each latitude and longitude combination.
for coordinate in coordinates:
    city = citipy.nearest_city(coordinate[0], coordinate[1]).city_name

    # If the city is unique, then we will add it to the cities list.
    if city not in cities:
        cities.append(city)
# Print the city count to confirm sufficient count.
len(cities)

627

In [6]:
# Create an empty list to hold the weather data.
city_data = []
# Print the beginning of the logging.
print("Beginning Data Retrieval     ")
print("-----------------------------")

# Create counters.
record_count = 1
set_count = 1

Beginning Data Retrieval     
-----------------------------


In [7]:
# Loop through all the cities in the list.
for i, city in enumerate(cities):

    # Group cities in sets of 50 for logging purposes.
    if (i % 50 == 0 and i >= 50):
        set_count += 1
        record_count = 1
    # Create endpoint URL with each city.
    city_url = url + "&q=" + city.replace(" ","+")

    # Log the URL, record, and set numbers and the city.
    print(f"Processing Record {record_count} of Set {set_count} | {city}")
    # Add 1 to the record count.
    record_count += 1
    
    # Run an API request for each of the cities.
    try:
        # Parse the JSON and retrieve data.
        city_weather = requests.get(city_url).json()
        # Parse out the needed data.
        city_lat = city_weather["coord"]["lat"]
        city_lng = city_weather["coord"]["lon"]
        city_max_temp = city_weather["main"]["temp_max"]
        city_humidity = city_weather["main"]["humidity"]
        city_clouds = city_weather["clouds"]["all"]
        city_wind = city_weather["wind"]["speed"]
        city_country = city_weather["sys"]["country"]
        # Convert the date to ISO standard.
        city_date = datetime.utcfromtimestamp(city_weather["dt"]).strftime('%Y-%m-%d %H:%M:%S')
        # Append the city information into city_data list.
        city_data.append({"City": city.title(),
                          "Lat": city_lat,
                          "Lng": city_lng,
                          "Max Temp": city_max_temp,
                          "Humidity": city_humidity,
                          "Cloudiness": city_clouds,
                          "Wind Speed": city_wind,
                          "Country": city_country,
                          "Date": city_date})

# If an error is experienced, skip the city.
    except:
        print("City not found. Skipping...")
        pass

# Indicate that Data Loading is complete.
print("-----------------------------")
print("Data Retrieval Complete      ")
print("-----------------------------")

Processing Record 1 of Set 1 | kaitangata
Processing Record 2 of Set 1 | maceio
Processing Record 3 of Set 1 | carnarvon
Processing Record 4 of Set 1 | port alfred
Processing Record 5 of Set 1 | albany
Processing Record 6 of Set 1 | sambava
Processing Record 7 of Set 1 | busselton
Processing Record 8 of Set 1 | torbay
Processing Record 9 of Set 1 | lompoc
Processing Record 10 of Set 1 | vaini
Processing Record 11 of Set 1 | ushuaia
Processing Record 12 of Set 1 | cape town
Processing Record 13 of Set 1 | mar del plata
Processing Record 14 of Set 1 | watsa
Processing Record 15 of Set 1 | cherskiy
Processing Record 16 of Set 1 | bethel
Processing Record 17 of Set 1 | avarua
Processing Record 18 of Set 1 | barrow
Processing Record 19 of Set 1 | hasaki
Processing Record 20 of Set 1 | hobart
Processing Record 21 of Set 1 | yerofey pavlovich
Processing Record 22 of Set 1 | punta arenas
Processing Record 23 of Set 1 | castro
Processing Record 24 of Set 1 | taolanaro
City not found. Skipping..

In [8]:
# Convert the array of dictionaries to a Pandas DataFrame.
city_data_df = pd.DataFrame(city_data)
city_data_df.head(10)

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Kaitangata,-46.2817,169.8464,45.93,59,100,7.54,NZ,2021-07-12 00:56:53
1,Maceio,-9.6658,-35.7353,74.64,94,0,3.44,BR,2021-07-12 00:56:54
2,Carnarvon,-24.8667,113.6333,68.07,94,75,13.80,AU,2021-07-12 00:56:54
3,Port Alfred,-33.5906,26.8910,64.89,28,0,8.84,ZA,2021-07-12 00:54:11
4,Albany,42.6001,-73.9662,66.02,94,100,3.62,US,2021-07-12 00:53:37
5,Sambava,-14.2667,50.1667,68.86,90,61,11.25,MG,2021-07-12 00:54:17
6,Busselton,-33.6500,115.3333,57.04,81,52,20.96,AU,2021-07-12 00:56:54
7,Torbay,47.6666,-52.7314,54.88,97,40,3.44,CA,2021-07-12 00:56:55
8,Lompoc,34.6391,-120.4579,89.74,74,20,5.01,US,2021-07-12 00:56:55
9,Vaini,-21.2000,-175.2000,78.96,89,20,10.36,TO,2021-07-12 00:56:55


In [10]:
new_column_order = [ "City" , "Country" , "Date" , "Lat" , "Lng" , "Max Temp" , "Humidity" , "Cloudiness" , "Wind Speed" ]
city_data_df = city_data_df[new_column_order]

In [11]:
city_data_df.head(10)

,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
0,Kaitangata,NZ,2021-07-12 00:56:53,-46.2817,169.8464,45.93,59,100,7.54
1,Maceio,BR,2021-07-12 00:56:54,-9.6658,-35.7353,74.64,94,0,3.44
2,Carnarvon,AU,2021-07-12 00:56:54,-24.8667,113.6333,68.07,94,75,13.80
3,Port Alfred,ZA,2021-07-12 00:54:11,-33.5906,26.8910,64.89,28,0,8.84
4,Albany,US,2021-07-12 00:53:37,42.6001,-73.9662,66.02,94,100,3.62
5,Sambava,MG,2021-07-12 00:54:17,-14.2667,50.1667,68.86,90,61,11.25
6,Busselton,AU,2021-07-12 00:56:54,-33.6500,115.3333,57.04,81,52,20.96
7,Torbay,CA,2021-07-12 00:56:55,47.6666,-52.7314,54.88,97,40,3.44
8,Lompoc,US,2021-07-12 00:56:55,34.6391,-120.4579,89.74,74,20,5.01
9,Vaini,TO,2021-07-12 00:56:55,-21.2000,-175.2000,78.96,89,20,10.36


In [12]:
# Create the output file (CSV).
output_data_file = "weather_data/cities.csv"
# Export the City_Data into a CSV.
city_data_df.to_csv(output_data_file, index_label="City_ID")